# Description

The notebook demonstrates how open-source solvers solve the DaoSwap problem.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
from typing import List, Tuple

import numpy as np

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import defi.dao_cross.optimize as ddacropt
import defi.dao_cross.order as ddacrord


<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [3]:
try:
    import pulp
except ImportError:
    !sudo /bin/bash -c "(source /venv/bin/activate; pip install pulp)"
    import pulp

In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-f728515a-aaa0-4817-8a8d-23a321969184.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.1' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask80_Implement_DaoSwap_optimization_problem_1'
  hash='af9f29ab9'
  # Last commits:
    * af9f29ab9 paul     Checkpoint                                                        (26 minutes ago) Tue Apr 18 19:57:14 2023  (HEAD -> CmTask80_Implement_DaoSwap_optimization_problem_1)
    * 098651322 Paul     Cm task4011 write dao swap white paper 11 (#147)                  (   2 hours ago) Tue Apr 18 18:14:10 2023  (origin/mast

# Test orders

In [5]:
def _generate_test_orders(
    actions: List[str],
    quantities: List[float],
    base_tokens: List[str],
    limit_prices: List[float],
    quote_tokens: List[str],
) -> List[ddacrord.Order]:
    """
    Create N `Order` instances using the inputs.

    See `Order` for params description.
    """
    # Use dummy values as the params are not relevant for the
    # optimization problem.
    timestamp = np.nan
    deposit_address = 1
    wallet_address = 1
    orders: List[ddacrord.Order] = []
    # TODO(Grisha): check that all lists are of the same length.
    for i in range(len(base_tokens)):
        order_i = ddacrord.Order(
            timestamp,
            actions[i],
            quantities[i],
            base_tokens[i],
            limit_prices[i],
            quote_tokens[i],
            deposit_address,
            wallet_address,
        )
        orders.append(order_i)
    return orders

In [6]:
_actions = ["buy", "buy", "sell", "sell", "buy", "buy", "sell", "sell"]
_quantities = [4, 2, 5, 3, 6, 2, 9, 1]
_base_tokens = ["BTC", "BTC", "BTC", "BTC", "ETH", "ETH", "ETH", "ETH"]
_quote_tokens = ["ETH", "ETH", "ETH", "ETH", "BTC", "BTC", "BTC", "BTC"]
_limit_prices = [3, 3.5, 1.5, 1.9, 0.6, 2, 0.1, 0.25]

def test1() -> None:
    """
    The limit price condition is True for all orders.
    """
    # Get inputs.
    prices = {"BTC": 3, "ETH": 6}
    limit_prices = [3, 3.5, 1.5, 1.9, 0.6, 2, 0.1, 0.25]
    test_orders = _generate_test_orders(
        _actions,
        _quantities,
        _base_tokens,
        _limit_prices,
        _quote_tokens,
    )
    return test_orders

In [7]:
orders = test1()

In [8]:
len(orders)

8

In [9]:
orders

[timestamp=2023-04-18 20:23:28.635157+00:00 action=buy quantity=4 base_token=BTC limit_price=3 quote_token=ETH deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.636897+00:00 action=buy quantity=2 base_token=BTC limit_price=3.5 quote_token=ETH deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.636998+00:00 action=sell quantity=5 base_token=BTC limit_price=1.5 quote_token=ETH deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.637044+00:00 action=sell quantity=3 base_token=BTC limit_price=1.9 quote_token=ETH deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.637086+00:00 action=buy quantity=6 base_token=ETH limit_price=0.6 quote_token=BTC deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.637127+00:00 action=buy quantity=2 base_token=ETH limit_price=2 quote_token=BTC deposit_address=1 wallet_address=1,
 timestamp=2023-04-18 20:23:28.637165+00:00 action=sell quantity=9 base_token=ETH limit_price=0.1 quote_toke

In [10]:
result = ddacropt.run_daoswap_solver(orders)

/venv/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
result

{'problem_status': 'Optimal',
 'problem_objective_value': 89.2,
 'q_pi_star': [4.0, 2.0, 5.0, 3.0, 6.0, 2.0, 9.0, 1.0],
 'q_tau_star': [12.0, 7.0, 7.5, 13.5, 3.6, 4.0, 0.9, 8.7],
 'solution_time_in_secs': 0.12}

In [12]:
effective_price = [result["q_tau_star"][i] / result["q_pi_star"][i] for i in range(len(orders))]

In [13]:
effective_price

[3.0, 3.5, 1.5, 4.5, 0.6, 2.0, 0.1, 8.7]